In [503]:
import re
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm_notebook
import sys
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from datetime import datetime
from lxml import etree, html as lhtml

In [504]:
authors_txt = 'hw3-data-retrieval/authors.txt'
book_links_txt = 'book_links.txt'
log_file_txt = 'log_file_1.txt'
site_name = 'https://www.respublica.ru'

In [505]:
def process_one_page(curr_address):
    r_get = requests.get(curr_address)
    if not (r_get.ok):
        with open(log_file_txt, 'a') as f:
            f.write(str(datetime.now())[:-7] + '\t:\t' + 'FAIL' + '\t:\t' + \
                    'Bad get request:\t' + curr_address + '\n')
        return
    author_soup = BeautifulSoup(r_get.text, 'lxml')
    books_descs = author_soup.find_all('a', itemprop='name')
    if books_descs is None:
        with open(log_file_txt, 'a') as f:
            f.write(str(datetime.now())[:-7] + '\t:\t' + 'FAIL' + '\t:\t' + \
                    'No books found:\t' + curr_address + '\n')
        return
    for i in range(len(books_descs)):
        with open(file_txt, 'a') as f:
            f.write('https://www.respublica.ru' + books_descs[i]['href'] + '\n')
    with open(log_file_txt, 'a') as f:
        f.write(str(datetime.now())[:-7] + '\t:\t' + 'SUCCESS' + '\t:\t' + curr_address + '\n')
        

def process_book_links(site_name, author_id, file_txt, log_file_txt):
    def find_the_number_of_iterations(author_soup):
        total_books_el = author_soup.find('div', class_="rd-listing-count")
        total_books = int(re.search('из \d+', total_books_el.text).group(0)[3:])
        total_pages = total_books / 22
        if total_books % 22 != 0:
            total_pages += 1
        return int(total_pages), total_books

    address_author_id = site_name + '/authors/' + author_id
    r_get = requests.get(address_author_id)
    if not (r_get.ok):
        with open(log_file_txt, 'a') as f:
            f.write(str(datetime.now())[:-7] + '\t:\t' + 'FAIL' + '\t:\t' + \
                    'Bad get request (on the main page):\t' + address_author_id + '\n')    
        return
    author_soup = BeautifulSoup(r_get.text, 'lxml')   
    total_pages, total_books = find_the_number_of_iterations(author_soup)

    pages = []
    for curr_page in range(2, total_pages+1):
        pages.append(address_author_id + '?page=' + str(curr_page))
    
    if total_pages != 1:        
        with ThreadPool(processes=total_pages-1) as pool:
            pool.map(process_one_page, pages)
    
    books_descs = author_soup.find_all('a', itemprop='name')
    if books_descs is None:
        with open(log_file_txt, 'a') as f:
            f.write(str(datetime.now())[:-7] + '\t:\t' + 'FAIL' + '\t:\t' + \
                    'No books found:\t' + address_author_id + '\n')
        return
    for i in range(len(books_descs)):
        with open(file_txt, 'a') as f:
            f.write(site_name + books_descs[i]['href'] + '\n')
    with open(log_file_txt, 'a') as f:
        f.write(str(datetime.now())[:-7] + '\t:\t' + 'SUCCESS' + '\t:\t' + address_author_id + '\n')
    pool.join()   

In [506]:
def count_lines(filename):
    with open(filename) as file:
        return sum(chunk.count('\n') for chunk in iter(lambda: file.read(), ''))

In [110]:
total_authors = count_lines(authors_txt) + 1
pbar = tqdm_notebook(total=total_authors)

with open(authors_txt, 'r') as f_read:
    line = f_read.readline()
    while line:
        author_id = line[:-1]
        with open(log_file_txt, 'a') as f_log:
            f_log.write(str(datetime.now())[:-7] + '\t:\t' + 'PROCESS STARTED' + '\t:\t' + author_id + '\n')
        process_book_links(site_name, author_id, book_links_txt, log_file_txt)
        pbar.update(1)
        line = f_read.readline()
with open(log_file_txt, 'a') as f_log:
    f_log.write(str(datetime.now())[:-7] + '\t:\t' + 'ALL PROCESSES COMPLETED' + '\n')

In [249]:
def process_page(url):
    pass

In [497]:
# def process_field(field_name):
def process_field(field_name, tree, card):
# def process_field(field_name, tree):
#     print(field_name)
    if field_name == 'Категория':
        el = tree.xpath('//div[@class="rd-page-breadcrumbs rd-page-product__breadcrumbs"]//text()')
        if len(el) == 0:
            return
        el = [re.search('\w+.*\w', c).group(0) for c in categories]
        card[field_name] = '; '.join(el)
    elif field_name == 'Название':
        el = tree.xpath('//h1[@class="rd-page-product__title"]/text()')
        if len(el) == 0:
            return
        card[field_name] = el[0]
    elif field_name == 'Автор':
        el = tree.xpath('//a[@itemprop="brand"]/text()')
        if len(el) == 0:
            return
        card[field_name] = el[0]
    elif field_name == 'ID':
        el = tree.xpath('//span[@itemprop="sku"]/text()')
        if len(el) == 0:
            return
        card[field_name] = el[0]
    elif field_name == 'Превью':
        el = tree.xpath('//a[@class="download-pdf"]/attribute::href')
        if len(el) == 0:
            return
        card[field_name] = site_name + el[0]
    elif field_name == 'Изображение':
        el = tree.xpath('//link[@rel="image_src"]/attribute::href')
        if len(el) == 0:
            return
        card[field_name] = el[0]
    elif field_name == 'Цена':
        el = tree.xpath('//span[@class="num"]/text()')
        if len(el) == 0:
            return
        card[field_name] = el[0]
    elif field_name == 'В наличии':
        el = tree.xpath('//span[@class="rd-page-product__buy-text"]/text()')
        if len(el) == 0:
            return
        if el[0] == 'Купить':
            card[field_name] = True
        else:
            card[field_name] = False
    elif field_name == 'Описание':
        el = tree.xpath('//div[@class="rd-page-product__desc-body"]/text()')
        if len(el) == 0:
            return
        card[field_name] = el[0]
    elif field_name == 'Характеристики':
        path_to_els = tree.xpath('//div[@class="rd-page-product__desc-params"]')[0]
        list_of_names = path_to_els.xpath('p[@class="rd-page-product__desc-param"]//text()')[0::3]
        list_of_els = path_to_els.xpath('p[@class="rd-page-product__desc-param"]//text()')[2::3]
        for i, el in enumerate(list_of_els):
            card[list_of_names[i]] = el   
    elif field_name == 'Цена (старая)':  
        el = tree.xpath('//span[@class="prev"]/text()')
        if len(el) == 0:
            return
        card[field_name] = re.search('\w+\w', el[0]).group(0)
    elif field_name == 'Рейтинг':      
        path_to_els = tree.xpath('//span[@itemprop="aggregateRating"]')
        if len(path_to_els) == 0:
            return
        # list_of_names = path_to_els.xpath('meta//attribute::itemprop')
        list_of_names = ['Число отзывов', 'Число оценок', 'Оценка']
        list_of_els = [path_to_els[0].xpath('meta[@itemprop="reviewCount"]/attribute::content')[0],
                       path_to_els[0].xpath('meta[@itemprop="ratingCount"]/attribute::content')[0],
                       path_to_els[0].xpath('meta[@itemprop="ratingValue"]/attribute::content')[0]]
        for i, el in enumerate(list_of_els):
            card[list_of_names[i]] = el    

In [500]:
url = 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/sovremennaya-proza/463224-bleyz'
url = 'https://www.respublica.ru/knigi/publitsistika/pisma-esse-intervyu/460219-kak-pisat-knigi'
r_get = requests.get(url)
tree = lhtml.fromstring(r_get.text)

In [501]:
card = dict()
field_list = ['Категория', 'Название', 'Автор', 'ID', \
              'Превью', 'Изображение', 'Цена', 'В наличии', \
              'Описание', 'Характеристики', 'Цена (старая)', 'Рейтинг']
for field in field_list:
    process_field(field, tree, card)

In [502]:
card

{'Категория': 'Книги; Художественная литература; Современная проза',
 'Название': 'Как писать книги',
 'Автор': 'Стивен Кинг',
 'ID': '460219',
 'Превью': 'https://www.respublica.ru/items/335822/download_preview_pdf',
 'Изображение': 'https://www.respublica.ru/uploads/00/00/00/58/li/large_cdb355864a8276cd.jpg',
 'Цена': '390',
 'В наличии': False,
 'Описание': 'Это – пожалуй, самая необычная из книг Стивена Кинга. Книга, в которой автобиографические, мемуарные мотивы соседствуют не только с размышлениями о писательском искусстве вообще, но и самыми настоящими «профессиональными советами тем, кто хочет писать, как Стивен Кинг».Как формируется писатель? Каковы главные «секреты» его нелегкого «ремесла»? Что, строго говоря, вообще необходимо знать и уметь человеку, чтобы его творения возглавляли международные списки бестселлеров?Вот лишь немногие из вопросов, на которые вы найдете ответы в этой книге. Вы действительно «хотите писать, как Стивен Кинг»? Тогда не пропустите эту книгу.',
 'ISB

In [298]:
r_get.text

'["/uploads/00/00/00/8j/5m/e52d6dcc0a528279.jpg","/uploads/01/00/00/8j/5n/8556aacd62e41478.jpg","/uploads/00/00/00/8j/5o/6a90ecc59bc711d3.jpg","/uploads/01/00/00/8j/5p/613e996579544995.jpg","/uploads/00/00/00/8j/5q/f8b24b544355f5bb.jpg","/uploads/01/00/00/8j/5r/08920beae6c3952e.jpg","/uploads/00/00/00/8j/5s/5c0df9eb4769a39f.jpg","/uploads/01/00/00/8j/5t/e2cfa046e157d0b6.jpg","/uploads/00/00/00/8j/5u/a6077d7fc2860233.jpg","/uploads/01/00/00/8j/5v/41e2d504d5d809b5.jpg","/uploads/00/00/00/8j/5w/ea56b47f33176b73.jpg","/uploads/01/00/00/8j/5x/ba0d7fa25d3a1d84.jpg","/uploads/00/00/00/8j/5y/0281604c3eacb05e.jpg","/uploads/01/00/00/8j/5z/7b0ec92348f63050.jpg","/uploads/00/00/00/8j/60/303a5405b691c05b.jpg"]'

In [ ]:
<div class="preview-button"><span class="eye-icon"></span>Смотреть фрагмент книги</div>

In [247]:
# url = 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/sovremennaya-proza/463224-bleyz'
# r_get = requests.get(url)
# author_soup = BeautifulSoup(r_get.text, 'lxml')
# books_descs = author_soup.find('div', class_='rd-page-breadcrumbs rd-page-product__breadcrumbs')

In [248]:
# books_descs.text

' Книги Художественная литература Современная проза'

In [239]:
def process_page(url):
    r_get = requests.get(url)
    if not (r_get.ok):
        with open(log_file_txt, 'a') as f:
            f.write(str(datetime.now())[:-7] + '\t:\t' + 'FAIL' + '\t:\t' + \
                    'Bad get request:\t' + curr_address + '\n')
        return
    author_soup = BeautifulSoup(r_get.text, 'lxml')
    
    
    print(author_soup)
    card = \
    {
        "ID": "444113",
        "URL": "https://www.respublica.ru/knigi/gumanitarnye-i-obschestvennye-nauki/filologiya/444113-pishi-sokraschay-kak-sozdavat-silnye-teksty",
        "Название": "Пиши, сокращай. Как создавать сильные тексты",
        "Автор": "Максим Ильяхов",
        "Превью": "https://www.respublica.ru/items/319412/download_preview_pdf",
        "Изображение": "https://www.respublica.ru/uploads/01/00/00/6e/ov/fullscreen_888e55eb57676e2d.jpg",
        "Описание": "Авторы на конкретных примерах показывают, что такое хорошо и что такое плохо в информационных, рекламных, журналистских и публицистических текстах. Как писать письма, на которые будут отвечать, и рассылки, от которых не будут отписываться. Как создавать действенные и не вульгарные рекламные объявления. Как излагать мысли кратко, ясно и убедительно: без языкового мусора, фальши и штампов. Следуя рекомендациям в книге, вы научитесь писать понятно, увлекать читателей и добиваться доверия. Это обязательная книга для копирайтеров, авторов и редакторов, а также дизайнеров, программистов, менеджеров, предпринимателей, руководителей, служащих и всех, кто использует текст в работе.",
        "Цена": 460,
        "Цена (старая)": 750,
        "В наличии": True,
        "Категория": "Книги; Гуманитарные и общественные науки; Филология",
        "Число отзывов": 29.0,
        "Число оценок": 29.0,
        "Оценка": 4.820689655172414,
        "ISBN": "978-5-9614-5967-8",
        "Издательство": "Альпина Паблишер",
        "Серия": "Маркетинг",
        "Обложка": "Мягкая",
        "Формат": "14 х 21",
        "Количество страниц": "440",
        "Год издания": "2017",
        "Язык": "Русский",
        "Направление": "Журналистика",
        "Страна-производитель": "Россия"
    }
    return card

In [240]:
import pandas as pd
book_links_txt = 'book_links_test.txt'

max_proc = 2
total_books = count_lines(book_links_txt)
pbar = tqdm_notebook(total=total_books)

result = []
with open(book_links_txt, 'r') as f_read:
    urls = []
    line = f_read.readline()
    while line:
        urls.append(line)
        if len(urls) == max_proc:
            with Pool(processes=max_proc) as pool:
                result += pool.map(process_page, urls)
            pool.join()
            urls = []   
            pbar.update(max_proc)
        line = f_read.readline()
    if len(urls) != 0:
        with Pool(processes=len(urls)) as pool:
            result += pool.map(process_page, urls)
        pbar.update(len(urls))

# result = list(map(process_page, urls))
df = pd.DataFrame(result)
df.sort_values(by=['ID'], inplace=True)
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml"><head><meta charset="utf-8"/><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"545c37ae31","applicationID":"10428862","transactionName":"clkNFRZWCFsAQhYKQlRbEE4XUQtA","queueTime":0,"applicationTime":173,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQYGVVdbGwIAUFNaDwcF",licenseKey:"545c37ae31",applicationID:"10428862"};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&cons

})();</script><script id="popmechanic-script" src="https://static.popmechanic.ru/service/loader.js?c=7246"></script></body></html><!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml"><head><meta charset="utf-8"/><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"545c37ae31","applicationID":"10428862","transactionName":"clkNFRZWCFsAQhYKQlRbEE4XUQtA","queueTime":0,"applicationTime":171,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQYGVVdbGwIAUFNaDwcF",licenseKey:"545c37ae31",applicationID:"10428862"};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof _

})();</script><script id="popmechanic-script" src="https://static.popmechanic.ru/service/loader.js?c=7246"></script></body></html>

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml"><head><meta charset="utf-8"/><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"545c37ae31","applicationID":"10428862","transactionName":"clkNFRZWCFsAQhYKQlRbEE4XUQtA","queueTime":0,"applicationTime":169,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQYGVVdbGwIAUFNaDwcF",licenseKey:"545c37ae31",applicationID:"10428862"};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof

})();</script><script id="popmechanic-script" src="https://static.popmechanic.ru/service/loader.js?c=7246"></script></body></html>


In [241]:
df

,ID,ISBN,URL,Автор,В наличии,Год издания,Издательство,Изображение,Категория,Количество страниц,...,Оценка,Превью,Серия,Страна-производитель,Формат,Цена,Цена (старая),Число отзывов,Число оценок,Язык
0,444113,978-5-9614-5967-8,https://www.respublica.ru/knigi/gumanitarnye-i...,Максим Ильяхов,True,2017,Альпина Паблишер,https://www.respublica.ru/uploads/01/00/00/6e/...,Книги; Гуманитарные и общественные науки; Фило...,440,...,4.82069,https://www.respublica.ru/items/319412/downloa...,Маркетинг,Россия,14 х 21,460,750,29.0,29.0,Русский
1,444113,978-5-9614-5967-8,https://www.respublica.ru/knigi/gumanitarnye-i...,Максим Ильяхов,True,2017,Альпина Паблишер,https://www.respublica.ru/uploads/01/00/00/6e/...,Книги; Гуманитарные и общественные науки; Фило...,440,...,4.82069,https://www.respublica.ru/items/319412/downloa...,Маркетинг,Россия,14 х 21,460,750,29.0,29.0,Русский
2,444113,978-5-9614-5967-8,https://www.respublica.ru/knigi/gumanitarnye-i...,Максим Ильяхов,True,2017,Альпина Паблишер,https://www.respublica.ru/uploads/01/00/00/6e/...,Книги; Гуманитарные и общественные науки; Фило...,440,...,4.82069,https://www.respublica.ru/items/319412/downloa...,Маркетинг,Россия,14 х 21,460,750,29.0,29.0,Русский


In [233]:
df = pd.DataFrame(result, columns =['ID', 'URL', 'Название', 'Автор', \
                                    'Превью', 'Изображение', 'Описание', 'Цена', \
                                    'Цена (старая)', 'В наличии', 'Категория', 'Число отзывов', \
                                    'Число оценок', 'Оценка', 'ISBN', 'Издательство', \
                                    'Серия', 'Обложка', 'Формат', 'Количество страниц', \
                                    'Год издания', 'Язык', 'Направление', 'Страна-производитель'])
# df

In [200]:
result = \
[{
    "ID": "444113",
    "URL": "https://www.respublica.ru/knigi/gumanitarnye-i-obschestvennye-nauki/filologiya/444113-pishi-sokraschay-kak-sozdavat-silnye-teksty",
},
{
    "ID": "884113",
    "URL": "https://www.respublica.ru/knigi/gumanitarnye-i-obschestvennye-nauki/filologiya/444113-pishi-sokraschay-kak-sozdavat-silnye-teksty",
}]

In [207]:
df = pd.DataFrame(result, columns =['ID', 'URL'])
df

,ID,URL
0,444113,https://www.respublica.ru/knigi/gumanitarnye-i...
1,884113,https://www.respublica.ru/knigi/gumanitarnye-i...


In [175]:
df = pd.DataFrame([result[:1]])
df

,0
0,"{'ID': '444113', 'URL': 'https://www.respublic..."


In [40]:
address_author_id = 'test_address_author_id/'

curr_addresses = []
for i in range(2,4):
    curr_addresses.append([address_author_id + '?page=' + str(i), 0, 's'])
curr_addresses

[['test_address_author_id/?page=2', 0, 's'],
 ['test_address_author_id/?page=3', 0, 's']]

In [10]:
st = '6 из 612323 товаров'
int(re.search('из \d+', st).group(0)[3:])

612323

In [23]:
def test_func(i):
    print(i[0])
    time.sleep(0.2)
        
def outer_test_func():
    values = [[0,1],[2,3],[4,5],[6,7]]
    with Pool(processes=10) as pool:
        pages = pool.map(test_func, values)
    pool.join()  
    print('flag')

In [24]:
outer_test_func()

0
4
2
6
flag


In [23]:
values = list(range(2,22,2))

In [24]:
%%time

for i in values:
    test_func(i,0)

2 1
4 1
6 1
8 1


KeyboardInterrupt: 

In [28]:
from itertools import product

In [27]:
%%time

with Pool(processes=10) as pool:
    pages = pool.map(test_func, values)
pool.join()   

TypeError: test_func() missing 1 required positional argument: 'x'

In [56]:
books_number_el = author_soup.find('div', class_="rd-listing-count")
int(re.search('\d+ товаров', books_number_el.text).group(0)[:-8])

198

In [64]:
import sys
f = open('text2.txt', 'a')

values = range(4)
pbar = tqdm_notebook(total=len(values)+1)
pbar.update(1)
for i in values:
    f.write(str(i)+'\n')
    pbar.update(1)
    sleep(0.5)
f.close()

In [22]:
values = range(32)
with tqdm_notebook(total=len(values)) as pbar:
    for i in values:
#         pbar.write('processed: %d' % (1 + i))
        pbar.update(1)
        sleep(0.05)

In [ ]:
curr_address = 
r_get = requests.get(curr_address)
if !r_get.ok:
    break
author_page = r_get.text
author_soup = BeautifulSoup(film_page, 'lxml')

In [67]:
temp_str

'/authors/15753?page=2'

In [ ]:
<a class="rd-listing-load-more" 
data-remote="true" 
href="/authors/15753?page=2&amp;ajax_append_page=true">
    <span class="rd-listing-load-more__text">
        Загрузить еще
    </span>
</a>